In [2]:
# things we need for NLP
import nltk
#from nltk.stem.lancaster import LancasterStemmer
#stemmer = LancasterStemmer()

from nltk.stem.rslp import RSLPStemmer
stemmer = nltk.stem.RSLPStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

# import our chat-bot intents file
import json
with open('intents_ptBR.json') as json_data:
    intents = json.load(json_data)

words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

44 documents
13 classes ['agradecimento', 'cumprimento', 'despedida', 'horario', 'pag_credito', 'pag_debito', 'pag_dinheiro', 'pedido', 'pizzas', 'pizzas_calabresa', 'pizzas_mussarela', 'pizzas_tipo', 'refrigerante']
69 unique stemmed words ['.', '1', '2', 'a', 'abert', 'abr', 'aceit', 'ai', 'ajud', 'algu', 'amex', 'até', 'aí', 'bel', 'bem', 'calabr', 'cart', 'coca-col', 'com', 'crédit', 'de', 'dinh', 'débit', 'e', 'est', 'eu', 'faz', 'funcion', 'gost', 'grat', 'guar', 'hoj', 'hor', 'horari', 'iss', 'log', 'loj', 'mais', 'mastercard', 'mei', 'muit', 'mussarel', 'o', 'obrig', 'oi', 'olá', 'pag', 'ped', 'pizz', 'poss', 'qu', 'qual', 'que', 'quer', 'refriger', 'sab', 'só', 'tchau', 'tem', 'tip', 'tud', 'um', 'uma', 'vai', 'val', 'vend', 'vis', 'voc', 'é']


In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print(train_x)
print(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 

In [9]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 50)
net = tflearn.fully_connected(net, 50)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=20, show_metric=False)
model.save('model.tflearn')

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

Training Step: 5999  | total loss: 0.25764 | time: 0.007s
| Adam | epoch: 2000 | loss: 0.25764 -- iter: 40/44
Training Step: 6000  | total loss: 0.23346 | time: 0.009s
| Adam | epoch: 2000 | loss: 0.23346 -- iter: 44/44
--
INFO:tensorflow:/Users/ericsakamoto/Documents/Projetos/Tensorflow/Chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
p = bow("A loja está aberta hoje?", words)
print (p)
print (classes)
print(model.predict([p]))

[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['agradecimento', 'cumprimento', 'despedida', 'horario', 'pag_credito', 'pag_debito', 'pag_dinheiro', 'pedido', 'pizzas', 'pizzas_calabresa', 'pizzas_mussarela', 'pizzas_tipo', 'refrigerante']
[[  4.06774692e-03   3.85953486e-02   4.30679676e-04   9.41979170e-01
    9.26134642e-04   1.68976246e-03   2.96581590e-07   5.49961851e-06
    5.65041974e-03   1.57978611e-05   4.83855046e-03   4.54680412e-04
    1.34594785e-03]]


In [8]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )